In [1]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
import gurobipy as gp
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style

c:\Users\HAOXUAN\miniconda3\envs\or\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [3]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'lpwp.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-18 17:52:14.884 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\lpwp.txt
2024-09-18 17:52:14.885 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\lpwp.txt
2024-09-18 17:52:14.887 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-18 17:52:14.887 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [4]:
class Code(BaseModel):
    reasoning: List[str]
    code: str     

In [5]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)
"""

sys_prompt = sys_prompt + example + "\nPlease finish the task think step by step."
print(sys_prompt)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEG

In [6]:
batch_size = 8
lp_reasoning_list = []
for idx in tqdm(range(0, len(questions), batch_size)):
    batch = questions[idx:idx+batch_size]
    
    tasks = [asyncclient.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        temperature=0,
        response_format=Code,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": f"QUESTION: {q}"}
        ]) for q in batch
    ]

    combined_responses = await asyncio.gather(*tasks)
    lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

100%|██████████| 36/36 [08:42<00:00, 14.52s/it]


In [7]:
filename = 'e2e_codegen_gpt_4o_mini_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(lp_reasoning_list, f)

In [8]:
codes = [lp_reasoning_list[i].code for i in range(len(lp_reasoning_list))]

In [9]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [10]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    cleand_code = '\n'.join(cleand_code)
    cleand_code = cleand_code.replace(')m', ')\nm')
    return cleand_code

In [11]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [12]:
pred_answers = []
for i, code_str in enumerate(codes):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-18 18:00:57.555 | ERROR    | __main__:<module>:10 - Error for question 0: name 'fish_per_sled_dog_trip' is not defined
2024-09-18 18:00:57.558 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-18 18:00:57.559 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-18 18:00:57.561 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-18 18:00:57.563 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-18 18:00:57.565 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-18 18:00:57.566 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-18 18:00:57.568 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-18 18:00:57.571 | INFO     | __main__:<module>:7 - question 8 obtain answer
2024-09-18 18:00:57.573 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-18 18:00:57.575 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-18 18:00:57.577 | INFO  

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-17
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x7e635da7
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+30   3.000000e+30   1.800000e+00      0s
       2    6.5000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.500000000e+04
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19

2024-09-18 18:00:57.789 | INFO     | __main__:<module>:7 - question 129 obtain answer
2024-09-18 18:00:57.791 | INFO     | __main__:<module>:7 - question 130 obtain answer
2024-09-18 18:00:57.793 | INFO     | __main__:<module>:7 - question 131 obtain answer
2024-09-18 18:00:57.794 | INFO     | __main__:<module>:7 - question 132 obtain answer
2024-09-18 18:00:57.796 | INFO     | __main__:<module>:7 - question 133 obtain answer
2024-09-18 18:00:57.797 | INFO     | __main__:<module>:7 - question 134 obtain answer
2024-09-18 18:00:57.798 | INFO     | __main__:<module>:7 - question 135 obtain answer
2024-09-18 18:00:57.800 | INFO     | __main__:<module>:7 - question 136 obtain answer
2024-09-18 18:00:57.801 | INFO     | __main__:<module>:7 - question 137 obtain answer
2024-09-18 18:00:57.803 | INFO     | __main__:<module>:7 - question 138 obtain answer
2024-09-18 18:00:57.805 | INFO     | __main__:<module>:7 - question 139 obtain answer
2024-09-18 18:00:57.806 | INFO     | __main__:<module>

In [13]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [14]:
codes[184]

'x = m.addVar(name="specialized_hours", vtype=gp.GRB.CONTINUOUS)\ny = m.addVar(name="common_hours", vtype=gp.GRB.CONTINUOUS)\nm.setObjective(100 * x + 72 * y, gp.GRB.MINIMIZE)\nm.addConstr(60 * x + 40 * y >= 10000)\nm.addConstr(60 * x >= 10000 / 3)'

In [15]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [16]:
error1, error2 = 1e-2, 1e-4
print(f"Under {error1} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error1)) / len(answers)}")
print(f"Under {error2} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error2)) / len(answers)}")

Under 0.01 error rate: the accuracy is 0.7361111111111112
Under 0.0001 error rate: the accuracy is 0.7256944444444444
